In [2]:
import numpy as np
import pandas as pd
import scipy.stats as st
import wandb

In [3]:
def get_metrics_dict(df : pd.DataFrame, run_notes : list,
                     metrics_dict : dict, convert_m_to_cm : bool=True,
                     verbose : bool=False) -> dict:
    notes_dict = {notes : metrics_dict.copy() for notes in run_notes}
    for notes in run_notes:
        # get each fold this notes was pretrained using the synthetic data
        notes_df = df.loc[df['Notes'] == notes]
        if notes_df.shape[0] != 5:
            if verbose:
                print(f'notes_df.shape[0] != 5 for notes {notes}, actual shape: {notes_df.shape}')
                print('getting 5 most recent runs')
            notes_df = notes_df.sort_values(by='Created', ascending=False).head(5)
            
        for metric in metrics_dict.keys():
            metric_values = notes_df[metric].values
            if verbose:
                print(f'{notes} {metric} {metric_values}')
            if convert_m_to_cm and ('mean_RMSE' in metric or 'mean_MAE' in metric):
                # convert from m^{-1} to cm^{-1}
                metric_values = metric_values * 1e-2
            # calculate mean and std for each metric
            notes_dict[notes][metric] = [np.mean(metric_values), np.std(metric_values)]
    return notes_dict

In [4]:
def print_single_tex_reslts_table(models_dict : dict, caption : str,
                                  label : str, metric_headers : str) -> None:
    metrics = list(list(models_dict.values())[0].keys())
    models = list(models_dict.keys())
    tex_table_string = f"""\\begin{{table*}}
    \\centering
    \\caption{caption}
    \\setlength{{\\tabcolsep}}{{3pt}}
    %\\begin{{tabular}}{{|p{{25pt}}|p{{75pt}}|p{{115pt}}|}}
    \\begin{{tabular}}{{|l|l|l|l|l|l|l|}}
    \\hline
    Model & """ + metric_headers + """ \\\\
    \\hline"""
    for model in models:
        row_name = '{' + model.replace('_',' ') + '}'
        tex_table_string += f"""\\multirow{{2}}{{*}}{row_name} & ${models_dict[model][metrics[0]][0]:.3f}$"""
        for metric in metrics[1:]:
            tex_table_string += f""" & ${models_dict[model][metric][0]:.3f}$"""
        tex_table_string += f""" \\\\ \n"""
        for metric in metrics:
            tex_table_string += f""" & $\\pm{models_dict[model][metric][1]:.3f}$"""
        tex_table_string += f""" \\\\ 
        \\hline"""
    tex_table_string += f"""\\end{{tabular}}
    \\label{label}
    \\end{{table*}}
    
    """
    
    print(tex_table_string)

In [8]:
def print_double_tex_reslts_table(models_dict_left : dict, header_left : str,
                                  models_dict_right : dict, header_right : str,
                                  caption : str, label : str, metric_headers : str,
                                  header_len : str='{4}') -> None:
    metrics_left = list(list(models_dict_left.values())[0].keys())
    metrics_right = list(list(models_dict_right.values())[0].keys())
    models = list(models_dict_left.keys()) # both dicts must have the same models
    tex_table_string = f"""\\begin{{table*}}
    \\centering
    \\caption{caption}
    \\label{{table}}
    \\setlength{{\\tabcolsep}}{{3pt}}
    %\\begin{{tabular}}{{|p{{25pt}}|p{{75pt}}|p{{115pt}}|}}
    \\resizebox{{\\textwidth}}{{!}}""" + """{""" + f"""\\begin{{tabular}}""" + """{{|l|"""

    for i in range(2*int(header_len[1])):
        tex_table_string += f"""l|"""
    tex_table_string += """}}"""
    
    tex_table_string += f"""
    \\hline
    \\multirow{{2}}{{*}}{{Model}} & \\multicolumn{header_len}{{|l|}}{header_left} & \\multicolumn{header_len}{{|l|}}{header_right} \\\\
    \\cline{{2-{str(int(header_len[1])*2+1)}}}
    & """ + metric_headers + """ & """ + metric_headers + """ \\\\
    \\hline"""
    for model in models:
        row_name = '{' + model.replace('_',' ') + '}'
        tex_table_string += f"""\\multirow{{2}}{{*}}{row_name} & ${models_dict_left[model][metrics_left[0]][0]:.3f}$"""
        for metric in metrics_left[1:]:
            tex_table_string += f""" & ${models_dict_left[model][metric][0]:.3f}$"""
        tex_table_string += f"""\n"""
        for metric in metrics_right:
            tex_table_string += f""" & ${models_dict_right[model][metric][0]:.3f}$"""
        tex_table_string += f""" \\\\ \n"""
        for metric in metrics_left:
            tex_table_string += f""" & $\\pm{models_dict_left[model][metric][1]:.3f}$"""
        tex_table_string += f"""\n"""
        for metric in metrics_right:
            tex_table_string += f""" & $\\pm{models_dict_right[model][metric][1]:.3f}$"""
        tex_table_string += f""" \\\\ 
        \\hline"""
    tex_table_string += f"""\\end{{tabular}}""" + """}""" + f"""
    \\label{label}
    \\end{{table*}}
    
    """
    
    print(tex_table_string)

In [9]:
if __name__ == "__main__":
    api = wandb.Api()

    # Optional: Filter by tags, state, etc. (see W&B API docs)
    FILTER = {}  # You can use {"state": "finished"} to get only completed runs

    # Project is specified by <entity/project-name>
    runs = api.runs("aisurrey_photoacoustics/MSOT_EDM2", filters=FILTER)

    run_dict_list = []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        run_dict_list.append(run.summary._json_dict)
        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        run_dict_list[-1].update(
            {k: v for k,v in run.config.items() if not k.startswith('_')}
        )
        # .name is the human-readable name of the run.
        run_dict_list[-1]['Name'] = run.name
        run_dict_list[-1]['Notes'] = run.notes
        run_dict_list[-1]['Created'] = run.createdAt

    df = pd.DataFrame(run_dict_list)

    # Preview the result
    print(df.head())

    columns = df.columns.values

        _runtime   _step    _timestamp               _wandb  \
0  256856.758797  468338  1.763070e+09  {'runtime': 256857}   
1  257677.029686  468338  1.763071e+09  {'runtime': 257677}   
2  257291.187475  468338  1.763071e+09  {'runtime': 257291}   
3  256903.996862  468338  1.763071e+09  {'runtime': 256904}   
4  256545.763539  468332  1.763070e+09  {'runtime': 256545}   

   bg_synthetic_test_mean_MAE  bg_synthetic_test_mean_PSNR  \
0                    2.516163                    26.678608   
1                    2.651931                    26.409498   
2                    2.607032                    26.664429   
3                    2.526328                    26.679118   
4                    2.499075                    26.805882   

   bg_synthetic_test_mean_R2  bg_synthetic_test_mean_RMSE  \
0                   0.739499                     3.093862   
1                  -0.199495                     3.241050   
2                   0.656033                     3.163144   
3   

In [10]:
if __name__ == "__main__":
    run_notes = [
        'e2eQPAT_Janeks_weights',
        'experimental_from_scratch_UNet_e2eQPAT',
        'e2eQPAT_fine_tune',
        'EDM2_fine_tune',
    ]
    inclusion_metrics_dict = {
        'inclusion_experimental_test_mean_RMSE' : [None]*5,
        'inclusion_experimental_test_mean_Rel_Err' : [None]*5,
        'inclusion_experimental_test_mean_PSNR' : [None]*5, 
        'inclusion_experimental_test_mean_SSIM' : [None]*5
    }
    bg_metrics_dict = {
        'bg_experimental_test_mean_RMSE' : [None]*5,
        'bg_experimental_test_mean_Rel_Err' : [None]*5,
        'bg_experimental_test_mean_PSNR' : [None]*5, 
        'bg_experimental_test_mean_SSIM' : [None]*5
    }    
    metric_headers = """RMSE (cm$^{{-1}}$) & Rel. Error (\\%) & PSNR & SSIM"""

    
    bg_metrics_models_dict = get_metrics_dict(
        df, run_notes, bg_metrics_dict, convert_m_to_cm=False
    )
    #bg_metrics_models_dict.update(get_metrics_dict(
    #    experimental_from_scratch_df, run_notes, bg_metrics_dict, convert_m_to_cm=False
    #))
    
    inclusion_metrics_models_dict = get_metrics_dict(
        df, run_notes, inclusion_metrics_dict, convert_m_to_cm=False
    )
    #inclusion_metrics_models_dict.update(get_metrics_dict(
    #    experimental_from_scratch_df, morun_notesdels, inclusion_metrics_dict, convert_m_to_cm=False
    #))

    print_double_tex_reslts_table(
        bg_metrics_models_dict,
        '{Background}',
        inclusion_metrics_models_dict, 
        '{Inclusions}',
        '{Experimental test dataset. Mean and standard deviation of 5 runs.}',
        '{tab:digimouse_test_metrics}',
        metric_headers
    )

\begin{table*}
    \centering
    \caption{Experimental test dataset. Mean and standard deviation of 5 runs.}
    \label{table}
    \setlength{\tabcolsep}{3pt}
    %\begin{tabular}{|p{25pt}|p{75pt}|p{115pt}|}
    \resizebox{\textwidth}{!}{\begin{tabular}{{|l|l|l|l|l|l|l|l|l|}}
    \hline
    \multirow{2}{*}{Model} & \multicolumn{4}{|l|}{Background} & \multicolumn{4}{|l|}{Inclusions} \\
    \cline{2-9}
    & RMSE (cm$^{{-1}}$) & Rel. Error (\%) & PSNR & SSIM & RMSE (cm$^{{-1}}$) & Rel. Error (\%) & PSNR & SSIM \\
    \hline\multirow{2}{*}{e2eQPAT Janeks weights} & $0.092$ & $48.706$ & $2.030$ & $0.004$
 & $0.708$ & $34.237$ & $8.849$ & $0.071$ \\ 
 & $\pm0.005$ & $\pm2.702$ & $\pm0.551$ & $\pm0.000$
 & $\pm0.028$ & $\pm2.524$ & $\pm0.604$ & $\pm0.024$ \\ 
        \hline\multirow{2}{*}{experimental from scratch UNet e2eQPAT} & $0.099$ & $52.690$ & $1.421$ & $0.004$
 & $0.707$ & $35.423$ & $8.542$ & $0.070$ \\ 
 & $\pm0.009$ & $\pm4.214$ & $\pm0.756$ & $\pm0.001$
 & $\pm0.072$ & $\pm5.389